In [44]:
### Importing ###
from numpy import exp, array, random, dot #imports linear algebra operations/types
import numpy as np #operations in numpy
from scipy import misc #misc. operations in scipy
import skimage
import math 
from __future__ import with_statement

In [45]:
### Get picture data ###
arr1 = np.divide(misc.imread('save_save_save_america02.png'),255) #read 1st image, divide by 255 for number between 0 and 1
arr1r = arr1[:,:,0] #reds
arr1g = arr1[:,:,1] #greens
arr1b = arr1[:,:,2] #blues
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [46]:
### Hyper-parameters ###
layerSize = 3 #number of filters in a layer
layerNum = 6 #number of layers
filterSize = 10 #size of filter
epochs = 10 #number of times trained
filters = [] #init filter
bias = 0.01
targets = np.ones((len(arr1r)*len(arr1r[0])*layerSize)) #target values

In [47]:
### Variables based on hyper-params ###
filterNum = layerSize**2 #number of filters per layer
totalFilterNum = filterNum*layerNum #total number of filters

In [48]:
### Set filters ###
def setFilters():
    global filters #Filters is a global variable
    filters = 0.01*random.random((layerNum, filterNum, filterSize, filterSize))-0.005 #Set filters to random 4d array
setFilters()
print(filters)

[[[[ -1.19659093e-03   5.10665481e-04  -7.38357490e-04 ...,
     -2.34530075e-03  -1.63792113e-03  -3.40708650e-04]
   [  2.61489610e-03   2.31521890e-03   1.46122307e-03 ...,
      2.33765308e-03   2.35623568e-03   4.35621253e-04]
   [  3.68480466e-03  -1.80084406e-03   1.91082865e-03 ...,
     -4.96788807e-05   4.68633549e-03  -2.22446805e-03]
   ..., 
   [  4.34329092e-03   1.01860223e-04   1.77847491e-03 ...,
      4.15275491e-03  -3.88328939e-03  -4.56101128e-03]
   [ -3.48336166e-03  -3.48144193e-03  -3.94328895e-03 ...,
      3.37196826e-03   4.61123044e-03  -2.52016597e-03]
   [  3.12794503e-03   2.74167336e-03  -1.01289599e-03 ...,
     -5.56352677e-04   1.71806492e-03   2.18534378e-03]]

  [[  3.79612039e-03  -2.96167848e-03   2.00795525e-03 ...,
     -1.19933400e-03  -1.24908927e-03   3.69462858e-03]
   [ -3.43014517e-03  -2.33039251e-03  -6.45627394e-04 ...,
     -2.36183259e-03  -6.15339529e-04  -4.32399537e-03]
   [  3.48787196e-04  -4.06393761e-03  -1.98652939e-03 ...,
 

In [49]:
### Helper functions ###
#Activation function (Leaky ReLU)
def activation(x, deriv=False, leak=-0.01):
    new_x=np.copy(array(x)) #Create a copy of x
    if deriv: #If you want to activate derivative
        #Calculation of derivative of leaky ReLU
        new_x[x > 0] = 1 
        new_x[x <= 0] = leak
        return new_x
    #Calculation of leaky ReLU
    new_x[x < 0] *= leak
    new_x = x * (x > 0)
    return new_x
#Sum with no negatives (not used)
'''
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
'''
#Apply cross-correlation for 1 index
def apply(filterArr, img, row, col):
    value = 0 #Init value
    halfFil = int(filterSize/2) #Half of filterSize
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0)) #Pad image with 0's
    for x in range(filterSize): 
        for y in range(filterSize): #Apply over all filter
            value += padImg[x+row][y+col]*filterArr[x][y] #Add image multiplied by filter
    return value
#Softmax function
def softmax(x):
#     exps = np.exp(x - x.max())
#     return exps / np.sum(exps)
    return np.exp(x) / np.sum(np.exp(x), axis=0) #Calculate softmax
print(apply(filters[1][1],arr1r,0,0))

0.00449840988712


In [50]:
### Apply filters ###
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])): #Apply over all image
            new_img[y][x] += activation(apply(filterArr,img,y,x))+bias#Add up all cross-correlations (with ReLU)
    return new_img

In [51]:
### Pooling ###
def pool(img, size):
    pad_w = size-len(img)%size
    pad_h = size-len(img[0])%size
    if pad_w == size:
        pad_w = 0
    if pad_h == size:
        pad_h = 0
    new_img = np.lib.pad(array(img), ((0,pad_w),(0,pad_h)), 'edge') #Pad image
    k = len(new_img)/size #Width of new image 
    l = len(new_img[0])/size #Height of new image
    return new_img.reshape(int(k),size,int(l),size).max(axis=(-1,-3)) #Arrange image into groups of size*size, then get max on axis

In [52]:
### Forward-propagation ###
def forward(img):
    outputLayers = [[img[:,:,0],img[:,:,1],img[:,:,2]]] #Get all the image 
    for num in range(layerNum):
        outputLayers.append([]) #Append a new layer
        for lay in range(layerSize):
            outputLayers[num+1].append(np.zeros((len(outputLayers[num][lay]), len(outputLayers[num][lay][0]))))
            # Add components of the layer
            outputLayers[num][lay] = pool(outputLayers[num][lay], 2)
            # Do pooling
        #lay1 is the layer index you are at and lay2 is the layer index you are going to
        for lay1 in range(layerSize):
            for lay2 in range(layerSize):
                newImage = outputLayers[num+1][lay2] #Get refrence to next layer
#                 newImage += bias
#                 print("S-O: ("+str(len(outputLayers))+", "+str(len(outputLayers[0]))+")")
#                 print("S-F: ("+str(len(filters))+", "+str(len(filters[0]))+")")
                applyFilters(outputLayers[num][lay1], newImage, filters[num][layerSize*lay1+lay2])
                #Apply corresponding filter
                if (filterNum*num + layerSize*lay1+lay2) % 20 == 0:
                    print(filterNum*num + layerSize*lay1+lay2)
#                 print(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten())) #Add flattened softmaxed array
    return outputLayers
out = forward(arr1)
print(out)

0
20
40
[[array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.23529412,  1.        ,  0.23529412,  1.        ,  0.23529412,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 1.        ,  0.23529412,  0.23529412,  1.        ,  0.23529412,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 0.23529412,  1.        ,  0.23529412,  1.        ,  0.35686275,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [ 0.92941176,  0.92941176,  0.92941176,  0.92941176,  0.92941176,
         0.92941176,  0.92941176,  0.92941176,  0.92941176,  0.92941176,
         0.92941176,  0.92941176,  0.92941176],
       [ 1.        ,

In [65]:
### Error + Back-propagation ###
error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets)) # negative sum of log(out) times target- cross-entropy
print("ERROR: "+str(error))
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
#        0 print("Filter S:"+str(len(filters)))
#         print(layerNum)
        for row in filters[layer-1][x]:
            c=0
            for w in row:
                new_grad[x][r][c]=-0.01*np.multiply(error,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer]),deriv=True),
                    w),
                    grad[x][r][c])))
                print(-0.01*np.multiply(error,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer]),deriv=True),
                    w),
                    grad[x][r][c]))))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[layer-1][x][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

ERROR: 3721.0135989
6
0.000960836738009
0.0023550108642
-0.00336132808642
-0.0173614267627
0.0398318379746
0.0382791706117
-0.0413700897374
0.0258366723856
0.0276769304786
-0.0079422634976
-0.00146704335135
-0.00198774385757
-0.0123829781378
0.00998499748328
-0.0109256086137
0.0461568715606
0.0319536840611
-0.0151872258576
0.0142242753004
0.00776189055973
-0.0417277546131
-0.0427999424389
0.00953135949158
-0.0141560922939
0.0442582235922
-0.0143153591383
-0.0407608813782
-0.0271373681393
0.0433600225319
-0.0198752689393
0.0379239762211
0.00971843577953
-0.00159629141873
-0.0416587614599
-0.0415870615669
-0.00104421539805
0.0184870947067
0.0450874779954
0.0119984936899
-0.00618144803623
-0.00157942300011
0.0123299345293
0.0144990802121
0.0187736757964
-0.00616362273595
-0.0228897050344
0.0130470151884
0.0245166914312
0.0238518860971
0.0350487783264
0.0132928432115
0.0124773480653
-0.0418296651249
-0.0179369953423
0.00353319291985
-0.0311924719227
0.033392692998
0.0271943912329
0.0323273

In [67]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'ignore'}

In [68]:
for x in range(epochs):
    out = forward(arr1)
    error = -np.sum(np.multiply(np.log10(out[len(out)-1]),targets))
    print(error)
#     print(filters)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))
    print(filters)

0
20
40
3695.06420789
6
-0.00600008585642
0.0170239158322
-0.0456122536231
0.0423091262056
0.0412263424699
0.0281321225161
0.0311494736881
0.0306046148399
-0.0142305185515
0.0313267912404
0.0129107956272
-0.0446677965164
-0.0141648002735
0.0205196672437
0.0409466941965
-0.0361446200236
0.0281864931994
0.0127624397856
0.0111683409844
-0.0131710398456
0.0195489046063
-0.00195841700056
0.0195030912889
-0.0371817054686
-0.032424314581
0.00598527774098
-0.00274811022241
-0.0201613707757
0.0172799888659
-0.0140929064221
-0.00849914299907
-0.00109257220394
-0.00426048170565
0.0179900214253
-0.0132082721547
0.0249737172537
-0.032280008473
0.0159165732918
-0.00212140271049
-0.0408929770251
0.0110538435305
0.0356260148462
0.0319752782516
0.0198774832855
0.0246944290538
-0.0148431935732
0.0348486915517
0.0231337059825
0.0294917178581
-0.0184216262157
0.0349549626111
-0.0335648460053
0.0204710320773
0.0154482868365
0.0351860256961
-0.00615997611071
-0.0281249954055
0.0171838365511
0.0322221332468


FloatingPointError: overflow encountered in exp